# NanoVer ASE Client / Server Example

This notebook walks through setting up a simulation with ASE and combining it with NanoVer to make an interactive molecular dynamics simulation. 
We then connect to the simulation from a client, so we can look at the data being produced and visualize it.

## Set up an ASE simulation

First, we set up an ASE simulation. We're going to do a simple lattice of copper particles under an EMT potential, but you can use [any supported calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html#module-ase.calculators)!

In [1]:
from ase import units
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [2]:
size = 2
# Set up a crystal
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol="Cu",
                          size=(size, size, size),
                          pbc=True)
# Describe the interatomic interactions with Effective Medium Theory
atoms.calc = EMT()
# Set the atomic momenta to 300 Kelvin. 
MaxwellBoltzmannDistribution(atoms, temperature_K=300 * units.kB)

Set up our molecular dynamics simulation, we'll run Langevin dynamics at room temperature

In [3]:
dyn = Langevin(atoms, timestep=1 * units.fs, temperature_K=300 * units.kB, friction=0.5)

Let's check it's working:

In [4]:
dyn.run(steps = 1)
dyn.get_number_of_steps()

1

In [5]:
# Note that this is the energy in eV, not kJ mol-1 (the standard units of ASE are different to those of OpenMM and NanoVer)
dyn.atoms.get_potential_energy()

-0.18180823716253158

In [6]:
from nanover.omni.ase import ASESimulation

nanover_imd = ASESimulation.from_ase_dynamics(dyn)

Let's run a couple of steps to make sure it's working

In [7]:
nanover_imd.dynamics.run(10)
nanover_imd.dynamics.nsteps

11

In [8]:
nanover_imd.dynamics.atoms.get_potential_energy()

-0.18179726385287687

## Set up the NanoVer iMD server

We fetch the NanoVer object we need to serve the dynamics. We provide a handy wrapper that understands ASE simulations and will serve the dynamics for you.

In [9]:
from nanover.omni import OmniRunner

Let's set up an IMD server: we set `port=0` to let the operating system pick a free port for us.

In [10]:
runner = OmniRunner.with_basic_server(nanover_imd, port=0) #Try using Shift+TAB+TAB to see what you can set up here.

**Note**: Be careful if you run the above cell multiple times without running `app_server.close()` (see below), as you will start multiple servers, which may be discovered if you use autoconnect. You can guard against this by swapping the cell above with:

```python
try:
    runner.close()
except NameError: # If the server hasn't been defined yet, there will be an error
    pass
runner = OmniRunner.with_basic_server(port=0)
```

A server is now ready to be discovered, we can see where it's running with the following:

In [11]:
print(f'{runner.app_server.name}: Running at {runner.app_server.address}:{runner.app_server.port}')

harrystroud: NanoVer iMD Server: Running at [::]:63903


The `[::]` means it is running on all available addresses, e.g. your WiFi and cabled access, and it's found an available port

Now, let's start the ASE simulation

In [12]:
runner.next()

In [13]:
# print the time to check dynamics is running
print(f'Simulation time: {nanover_imd.dynamics.get_time()}, ({nanover_imd.dynamics.nsteps} steps)')

Simulation time: 1.7680850619235313, (18 steps)


Ok, it's working, so let's leave it running dynamics in background thread. This is what happens by default when you call `run`

That's it, your simulation is running and interactive! If you connect to it from iMD-VR, you'll see something like:

![NanoVer ASE EMT](./images/nanover_ase_emt.png)

You can also visualize it live in the notebook, (check out this [example with NGLView](../basics/nanover_nglview.ipynb)).

## Start an IMD client

Let's have a look at the data that's being produced here, by connecting a python client

In [14]:
from nanover.app import NanoverImdClient
client = NanoverImdClient.connect_to_single_server(port=runner.app_server.port)

Subscribe to the stream of frames. The server will try to send 30 frames per seconds, if it produces more frames, some frames will be skipped to maintain the cadence.

In [15]:
client.subscribe_to_frames()

Let's grab a frame

In [16]:
client.wait_until_first_frame()
frame = client.current_frame

In [17]:
frame.particle_count

32

In [18]:
# Print the potential energy, here in kJ mol-1 as we are accessing it via NanoVer
frame.potential_energy # use TAB to see what's available!

-17.533980241570806

We can also print out the raw underlying data that is sent to VR, and see all the simulation data that is being transmitted

In [19]:
# view the latest frame
client.current_frame.raw

values {
  key: "system.simulation.counter"
  value {
    number_value: 0
  }
}
values {
  key: "server.timestamp"
  value {
    number_value: 71745.708391166
  }
}
values {
  key: "residue.count"
  value {
    number_value: 1
  }
}
values {
  key: "particle.count"
  value {
    number_value: 32
  }
}
values {
  key: "energy.potential"
  value {
    number_value: -17.532057455938844
  }
}
values {
  key: "energy.kinetic"
  value {
    number_value: 0.010858480390873912
  }
}
values {
  key: "chain.count"
  value {
    number_value: 1
  }
}
arrays {
  key: "system.box.vectors"
  value {
    float_values {
      values: 0.722
      values: 0
      values: 0
      values: 0
      values: 0.722
      values: 0
      values: 0
      values: 0
      values: 0.722
    }
  }
}
arrays {
  key: "residue.names"
  value {
    string_values {
      values: "ASE"
    }
  }
}
arrays {
  key: "residue.ids"
  value {
    string_values {
      values: "1"
    }
  }
}
arrays {
  key: "residue.chains"
  v

Now, let's visualize the frame directly with ASE. First we convert the frame back into ASE atoms, and view it. The first frame received by a client always has the topology information

In [20]:
from nanover.ase.converter import frame_data_to_ase
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)

We can run this to update the latest positions from the frame (note that NanoVer uses **nanometers** for positions. [Why??](http://docs.openmm.org/6.2.0/userguide/theory.html#units))

In [21]:
import numpy as np
# set the positions to match the latest frame data
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

ASE has a cool little 2D visualizer

In [22]:
from ase.visualize import view
view(atoms)

<Popen: returncode: None args: ['/opt/homebrew/Caskroom/miniforge/base/envs/...>

And a 3D visualizer! (Check your browser)

In [23]:
view(atoms, viewer='x3d')

# Gracefully terminate!!

It is very important to close your servers, otherwise they'll get in the way by blocking ports, and your clients may autoconnect to the wrong server!

In [24]:
runner.close()

Note that outside of a notebook, you can use `with` statements to let NanoVer clean up after itself:

In [25]:
with OmniRunner.with_basic_server(ASESimulation.from_ase_dynamics(dyn), port=0) as runner:
    runner.next()

# Next Steps

This notebook showed a toy example with ASE. 

* Run bigger molecular mechanics simulations with OpenMM. The [nanotube](./ase_openmm_nanotube.ipynb) and [neuraminidase](./ase_openmm_neuraminidase.ipynb) examples show how to set up simulations with OpenMM.
* To understand what's going on under the hood, consider starting with the [NanoVer frame explained](../fundamentals/frame.ipynb) example. 